In [ ]:
!pip install openai langchain chromadb tiktoken faiss-cpu -qq
!pip install matplotlib seaborn -qq
!pip install sentence-transformers -qq
!pip install PyPDF2 -qq

In [16]:
import os
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

In [9]:
model = SentenceTransformer("intfloat/multilingual-e5-base")
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [12]:
def embed_chunks(texts, model):
    embeddings = model.encode(chunks, 
                             show_progress_bar = True,
                             normlize_embeddings = True)
    return embeddings

In [14]:
def kb_vectors(kb_chunks , model = model):
    with open(kb_chunks, 'r', encoding = 'utf-8') as file:
        chunks = json.load(file)

    texts = [chunk['text'] for chunk in chunks]
    ids = [chunk['id'] for chunk in chunks]

    embeddings = embed_chunks(texts, model)

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    index_file = 'kb_vector.index'
    meta_file = 'kb_vector.meta.json'

    faiss.write_index(index, index_file)

    with open(meta_file, 'w', encoding = 'utf-8') as file:
        json.dump(ids, file, ensure_ascii = False, indent = 2)

    return index_file, meta_file

In [15]:
index_file, meta_file = kb_vectors(kb_chunks = '/kaggle/input/kb-chunks/kb_chunks.json')

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [40]:
def load_index(index_file="/kaggle/working/kb_vector.index", meta_file="/kaggle/input/kb-chunks/kb_chunks.json"):
    # تحميل الفهرس (binary)
    index = faiss.read_index(index_file)

    # تحميل الـ IDs (json)
    with open(meta_file, "r", encoding="utf-8") as f:
        ids = json.load(f)

    return index, ids

def search(query, top_k=3, index_file="/kaggle/working/kb_vector.index", meta_file="/kaggle/input/kb-chunks/kb_chunks.json", model_name="intfloat/multilingual-e5-base"):
    # تحميل الفهرس والـ IDs
    index, chunks = load_index(index_file, meta_file)

    # تحميل الموديل
    model = SentenceTransformer(model_name)

    # حساب embedding للسؤال
    query_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    # البحث
    distances, indices = index.search(query_emb, top_k)

    # تجهيز النتائج
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "id": chunks[idx]["id"],
            "text": chunks[idx]["text"],     # ✅ النص نفسه
            "source": chunks[idx]["source"], # ✅ المصدر (الملف الأصلي)
            "score": float(distances[0][i])
        })

    return results


In [43]:
query = "ادارة الوقت في الاسلام"
results = search(query, top_k=3)

print("\n🔎 Search Results:")

for r in results:
    print(f"\n📄 Source: {r['source']}")
    print(f"🔗 Chunk ID: {r['id']}")
    print(f"💡 Score: {r['score']:.4f}")
    print(f"📝 Content: {r['text'][:]}...")  # عرض أول 300 حرف

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔎 Search Results:

📄 Source: ../kb\0837b3ae6ae76f97f7eb1532ac21e01f.txt
🔗 Chunk ID: 0837b3ae6ae76f97f7eb1532ac21e01f.txt_6
💡 Score: 0.8539
📝 Content: in the Quran, the Islamic perspective on time management, and examples of wasting time in daily life and how to avoid it. Whenever we talk about time, we always come back to the Quranic verse from Surah Al Asar where Allah says: وَالْعَصْرِ. إِنَّ الْإِنسَانَ لَفِي خُسْرٍ. إِلَّا الَّذِينَ آمَنُوا وَعَمِلُوا الصَّالِحَاتِ وَتَوَاصَوْا بِالْحَقِّ وَتَوَاصَوْا بِالصَّبْرِ ‘By time, Indeed, mankind is in loss, Except for those who have believed and done righteous deeds and advised each other to...

📄 Source: ../kb\0837b3ae6ae76f97f7eb1532ac21e01f.txt
🔗 Chunk ID: 0837b3ae6ae76f97f7eb1532ac21e01f.txt_2
💡 Score: 0.8482
📝 Content: and inspiring framework for time management, transforming fleeting moments into opportunities for profound personal growth and lasting impact. Related: The Top Quran Translations and Commentaries on Amazon In Islam, t